# Check out simple pangeo-forge recipe

This notebook demonstrates how the default result of pangeo-forge `StoreToZarr` results coordinates being chunked.

In [1]:
import apache_beam as beam
import fsspec
import h5py
import os
from pangeo_forge_recipes.patterns import FilePattern, ConcatDim
from pangeo_forge_recipes.transforms import OpenURLWithFSSpec, OpenWithXarray, StoreToZarr
import re
import s3fs
from tempfile import TemporaryDirectory
import xarray as xr

In [2]:
!pip show pangeo_forge_recipes

Name: pangeo-forge-recipes
Version: 0.9.1.dev229+gb73cc39
Summary: Pipeline tools for building and publishing analysis ready datasets.
Home-page: https://github.com/pangeo-forge/pangeo-forge-recipes
Author: 
Author-email: 
License: Apache
Location: /srv/conda/envs/notebook/lib/python3.10/site-packages
Requires: cftime, dask, distributed, fsspec, h5netcdf, h5py, intake, intake-xarray, kerchunk, mypy-extensions, netcdf4, numcodecs, setuptools, xarray, zarr
Required-by: 


## 1. Set parameters

In [3]:
#parameters
temporal_resolution = "daily"
model = "GISS-E2-1-G"
variable = "tas"
anon=True

In [4]:
# Initiate fsspec filesystems for reading and writing
s3_path = f"s3://nex-gddp-cmip6/NEX-GDDP-CMIP6/{model}/historical/r1i1p1*/{variable}/*"
fs_read = fsspec.filesystem("s3", anon=anon, skip_instance_cache=False)
fs_write = fsspec.filesystem("")

In [5]:
# Retrieve list of available months
files_paths = fs_read.glob(s3_path)
print(f"{len(files_paths)} discovered from {s3_path}")

65 discovered from s3://nex-gddp-cmip6/NEX-GDDP-CMIP6/GISS-E2-1-G/historical/r1i1p1*/tas/*


In [6]:
files_paths[0]

'nex-gddp-cmip6/NEX-GDDP-CMIP6/GISS-E2-1-G/historical/r1i1p1f2/tas/tas_day_GISS-E2-1-G_historical_r1i1p1f2_gn_1950.nc'

# 2. Test we can open the files

In [7]:
fs_s3 = s3fs.S3FileSystem(anon=True)
filepath = f's3://{files_paths[0]}'
f = fs_s3.open(filepath, mode='rb')
ds = xr.open_dataset(f)
ds

<xarray.Dataset>
Dimensions:  (time: 365, lat: 600, lon: 1440)
Coordinates:
  * time     (time) object 1950-01-01 12:00:00 ... 1950-12-31 12:00:00
  * lat      (lat) float64 -59.88 -59.62 -59.38 -59.12 ... 89.38 89.62 89.88
  * lon      (lon) float64 0.125 0.375 0.625 0.875 ... 359.1 359.4 359.6 359.9
Data variables:
    tas      (time, lat, lon) float32 ...
Attributes: (12/23)
    downscalingModel:      BCSD
    activity:              NEX-GDDP-CMIP6
    contact:               Dr. Rama Nemani: rama.nemani@nasa.gov, Dr. Bridget...
    Conventions:           CF-1.7
    creation_date:         2021-10-04T18:41:40.796912+00:00
    frequency:             day
    ...                    ...
    history:               2021-10-04T18:41:40.796912+00:00: install global a...
    disclaimer:            This data is considered provisional and subject to...
    external_variables:    areacella
    cmip6_source_id:       GISS-E2-1-G
    cmip6_institution_id:  NASA-GISS
    cmip6_license:         CC-BY-SA 4.0

## Note: tas is chunked by time, but the time coordinate itself is not.

In [16]:
with fs_read.open(filepath, 'rb') as s3_file:
    with h5py.File(s3_file, 'r') as hdf5_file:
        print(hdf5_file['tas'].chunks)
        print(hdf5_file['time'].chunks)        

(1, 600, 1440)
(365,)


# 3. Set the destination

In [8]:
def format_function(time):
    pattern = r"\b\d{4}\b"
    return re.sub(pattern, str(time), filepath)

years = list(range(1950, 1952))
time_dim = ConcatDim("time", keys=years)

pattern = FilePattern(format_function, time_dim, file_type="netcdf4")
pattern = FilePattern.prune(pattern, nkeep=2)

In [9]:
pattern.combine_dims

(ConcatDim(name='time', nitems_per_file=None),)

In [10]:
td = TemporaryDirectory()
target_root = td.name
store_name = "test-cmip6.zarr"
target_path = os.path.join(target_root, store_name)
target_path

'/tmp/tmpn4ks2n0h/test-cmip6.zarr'

# 4. Set different target chunks

In [11]:
global_target_chunks = { 'lat': ds.lat.shape[0], 'lon': ds.lon.shape[0], 'time': 1 }
global_target_chunks

{'lat': 600, 'lon': 1440, 'time': 1}

# 5. Run pipeline

In [12]:
transforms = (
    beam.Create(pattern.items())
    | OpenURLWithFSSpec(open_kwargs={'anon': True})
    | OpenWithXarray(file_type=pattern.file_type)
    | StoreToZarr(
        store_name=store_name,
        target_root=target_root,
        combine_dims=pattern.combine_dim_keys,
        target_chunks=global_target_chunks,
    )
)
with beam.Pipeline() as p:
    p | transforms

## Coordinates are chunked - why?

A similar result will happen if we chunk the variable on lat and lon coordinates (the coordinates themselves will be chunked).

In [18]:
!ls {target_path}/time/ | head

0
1
10
100
101
102
103
104
105
106
0
